In [4]:
import gdown

url = 'https://drive.google.com/uc?id=1moaGNpmZjjN0XH0dcTOybEz8NVS00x-6'
output = 'train_dataset.txt'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=19ajg9r-QMvDBuVDxBne4Y-vIzSqauAHS'
output = 'test_dataset.txt'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1moaGNpmZjjN0XH0dcTOybEz8NVS00x-6
To: /content/train_dataset.txt
100%|██████████| 17.0M/17.0M [00:00<00:00, 103MB/s] 
Downloading...
From: https://drive.google.com/uc?id=19ajg9r-QMvDBuVDxBne4Y-vIzSqauAHS
To: /content/test_dataset.txt
100%|██████████| 4.25M/4.25M [00:00<00:00, 77.3MB/s]


'test_dataset.txt'

In [5]:
import csv
from scipy.sparse import csr_matrix

num_records = 0
record = []
with open('train_dataset.txt', 'r', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter='\t', quoting=csv.QUOTE_ALL, quotechar='"')
    next(reader) # Skip the first line
    for row in reader:
        if len(row) == 4:
            record.append(row)

In [6]:
unique_users = list(set([row[0] for row in record]))
unique_items = list(set([row[1] for row in record]))

# Create mappings from user_id and item_id to row and column indices
user_mapping = {user: index for index, user in enumerate(unique_users)}
item_mapping = {item: index for index, item in enumerate(unique_items)}

# Map user_id and item_id to row and column indices
record = [[user_mapping[row[0]], item_mapping[row[1]], row[2], row[3]] for row in record]

# Separate the ratings and review texts
ratings = [float(row[2]) for row in record]

# Create a R matrix for the ratings
r = csr_matrix((ratings, ([record[i][0] for i in range(len(record))], [record[i][1] for i in range(len(record))])))
R_matrix = r.toarray()
print(f'Number of records: {num_records}')
print(f'R matrix: \n{R_matrix}')

Number of records: 23038
R matrix: 
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# **Load Item Embeddings**

In [7]:
import torch
item_embeddings = torch.load('item_embeddings.pt', map_location=torch.device('cpu'))

In [ ]:
display(item_embeddings)

In [9]:
from sklearn.decomposition import PCA
import numpy as np

item_embeddings_np = {}
for item_id, embedding in item_embeddings.items():

    # Flatten the tensor and convert it to a numpy array
    item_embeddings_np[item_id] = embedding.view(-1).cpu().detach().numpy()



embeddings_np = np.array(list(item_embeddings_np.values()))



In [10]:
print(embeddings_np.shape)

(733, 768)


In [11]:
print(embeddings_np)

[[ 0.04237541 -0.08171102  0.32029814 ...  0.03018228 -0.04695213
   0.06047632]
 [ 0.05445423 -0.10293787  0.32472286 ... -0.09971305 -0.05402731
   0.09488987]
 [-0.06542997 -0.147529    0.36194992 ...  0.01217275 -0.0115719
   0.09496435]
 ...
 [ 0.05231054 -0.11189589  0.3390529  ... -0.09437604 -0.06007697
   0.06164911]
 [ 0.0610081  -0.11780063  0.36431956 ... -0.05020854 -0.08578517
   0.0810637 ]
 [ 0.00204834 -0.14168464  0.34443936 ... -0.073659    0.00549071
  -0.00100103]]


# **B)Concat Q Matrix to reduced matrix (with pca)**

In [12]:
import numpy as np


Matrix_P = np.load('P.npy')
Matrix_Q = np.load('Q.npy')

In [13]:
print(Matrix_Q.shape)

(733, 64)


In [26]:
concat_matrix = np.concatenate((embeddings_np, Matrix_Q), axis=1)
print(concat_matrix.shape)

(733, 832)


## **PCA**

In [27]:
# do pca
pca = PCA(n_components=64)

reducded_embedding= pca.fit_transform(concat_matrix)


In [16]:
print(reducded_embedding.shape)

(733, 64)


In [17]:
Matrix_Q = reducded_embedding

# **Rating Prediction**

In [19]:
import numpy as np
from tqdm import tqdm

def predict_rating(P , Q , i, u):

        return Q[i, :].dot(P[u, :].T)

In [20]:
import pandas as pd

# Load test dataset without header
test_data = pd.read_csv('test_dataset.txt', sep='\t', header=None, skiprows=1)

test_data.columns = ['user_id', 'item_id', 'rate', 'review_text']

ground_truth = test_data.groupby('user_id')['item_id'].apply(list).reset_index()
display(ground_truth)
ground_truth.columns = ['user_id', 'interacted_items']


,user_id,item_id
0,A100WO06OQR8BQ,"[B009FKNGGQ, B003U4LI7W, B007JT7ARQ, B001ECQ55..."
1,A100ZQDV7L8PVV,"[B005TI7LHS, B009XRF9M0]"
2,A105S56ODHGJEK,"[B00GTC02LA, B00AWLB9G6, B00AE07CRA, B006L6A06..."
3,A10E3F50DIUJEE,"[B009YSSLAU, B00GTBZI6A]"
4,A10M94ASQEBL56,"[B00GTBZWPW, B00AO4E9L8]"
...,...,...
1335,AYWHCM0TJ4737,"[B00AE07932, B00GTC1JHQ]"
1336,AZ26CDSJ363AH,"[B0073P01RC, B00BB8ZHJY, B00AE0790U]"
1337,AZA595ZPIG240,"[B001ECQ4YO, B00AO4E9MC, B00D6EDGYE, B00AE07FU..."
1338,AZFHSPEZUPGD2,"[B002UUT3YM, B0054MSBZA, B00639DLV2, B009YSSLA..."


In [21]:
all_items = set(unique_items) | set(test_data['item_id'])

train_interacted_items = {user: set(item[1] for item in record if item[0] == user_mapping[user]) for user in unique_users}

predicted_ratings = {}
for user in set(test_data['user_id']):
    user_idx = user_mapping[user]
    predicted_ratings[user] = {item: predict_rating(Matrix_P , Matrix_Q , item_mapping[item], user_idx) for item in all_items if item not in train_interacted_items[user]}



# **Evaluation**

In [22]:
from sklearn.metrics import recall_score

def calculate_recall(predicted_ratings, ground_truth, topk=20):
    recall_scores = []
    for user, ratings in predicted_ratings.items():
        # Get the top 20 items predicted for the user
        top_20_predicted_items = sorted(ratings.items(), key=lambda x: x[1], reverse=True)[:topk]

        # Get the actual items interacted by the user
        actual_items = ground_truth[ground_truth['user_id'] == user]['interacted_items'].values[0]

        actual_items_binary = [1 for item in actual_items]

        # Convert the items to binary format (1 if the item is in the top 20 predicted items, 0 otherwise)
        predicted_ratings_binary = [1 if item in dict(top_20_predicted_items) else 0 for item in actual_items]

        # Calculate the recall for the user
        recall_score_user = recall_score([actual_items_binary], [predicted_ratings_binary], average = 'weighted')
        recall_scores.append(recall_score_user)


    return np.mean(recall_scores)

In [23]:
from sklearn.metrics import ndcg_score

def calculate_ndcg(predicted_ratings , ground_truth, topk=20):

    ndcg_scores = []
    for user, ratings in predicted_ratings.items():

        # Get the top 20 items predicted for the user
        top_20_predicted_items = sorted(ratings.items(), key=lambda x: x[1], reverse=True)[:topk]

        # Get the actual items interacted by the user
        actual_items = list(ground_truth[ground_truth['user_id'] == user]['interacted_items'].values[0])

        actual_items_binary = [1 for item in actual_items]


        predicted_ratings_binary = [dict(top_20_predicted_items).get(item, 0) if item in dict(top_20_predicted_items) else 0 for item in actual_items]


        actual_items_binary = np.asarray([actual_items_binary])
        predicted_ratings_binary = np.asarray([predicted_ratings_binary])

        # Calculate the NDCG for the user
        ndcg_score_user = ndcg_score(actual_items_binary, predicted_ratings_binary)
        ndcg_scores.append(ndcg_score_user)


    return np.mean(ndcg_scores)

In [24]:
from scipy import stats
def calculate_rankcorrelation(predicted_ratings, ground_truth):

    spearman_scores = []
    for user, ratings in predicted_ratings.items():

        predicted_items = sorted(ratings.items(), key=lambda x: x[1], reverse=True)


        # Get the actual items interacted by the user
        actual_items = list(ground_truth[ground_truth['user_id'] == user]['interacted_items'].values[0])

        actual_items_ranking = list(range(1, len(actual_items) + 1)) # Assign ranks to actual items



      # Assign ranks to actual items based on their scores in the predicted list

        actual_items_ranking_in_predicted_list = [next((index + 1 for index, item in enumerate(predicted_items) if item[0] == target), None)  for target in actual_items]

        # Calculate the sperman correlation for the user
        spearman_score_user ,_ = stats.spearmanr(actual_items_ranking, actual_items_ranking_in_predicted_list )
        spearman_scores.append(spearman_score_user)

    return np.mean(spearman_scores)

In [25]:
print("recall score is : " , calculate_recall(predicted_ratings, ground_truth))

print("rank correlation(spearman) is : " , calculate_rankcorrelation(predicted_ratings , ground_truth) )

print("ndcg score is : " , calculate_ndcg(predicted_ratings, ground_truth))


recall score is :  0.02820443859622964
rank correlation(spearman) is :  0.028703421845060202
ndcg score is :  1.0
